In [3]:
import os
import pandas as pd
from PIL import Image
Image.MAX_IMAGE_PIXELS=None

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

from openslide import OpenSlide
import numpy as np
import cv2
import os
from time import time

In [4]:
def cellclass(cnt,dl,rsfw_wsi2dl,rsfh_wsi2dl):
    celltype = dl.getpixel((cnt[0]//rsfw_wsi2dl,cnt[1]//rsfh_wsi2dl))
    return celltype

In [5]:
def cntarea(cnt):
    cnt = np.array(cnt)
    area = cv2.contourArea(cnt)
    return area

def cntperi(cnt):
    cnt = np.array(cnt)
    perimeter = cv2.arcLength(cnt,True)
    return perimeter

def cntMA(cnt):
    cnt = np.array(cnt)
    #Orientation, Aspect_ratio
    (x,y),(MA,ma),orientation = cv2.fitEllipse(cnt)
    return np.max((MA,ma)),np.min((MA,ma)),orientation

In [49]:
#jsonsrc = r"\\10.99.68.178\andreex\students\Eduarda PDAC tissue composition\3D_annotations\TC_039\hovernet_out\json"
#dlsrc = r"\\10.99.68.178\andreex\students\Eduarda PDAC tissue composition\3D_annotations\TC_039\10x\classification_5_7_2023"
#wsisrc = r"\\10.99.68.178\andreex\students\Eduarda PDAC tissue composition\3D_annotations\TC_039"

jsonsrc = r"\\10.99.68.178\andreex\data\monkey fetus\gestational 40\StarDist_11_21_23\json"
m1src = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\5x\cropped_images\classification_MODEL1_6_21_2023_all_annotations'
m2src = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\all_classifications'
wsisrc = r"\\10.99.68.178\andreex\data\monkey fetus\gestational 40"

jsons = sorted([_ for _ in os.listdir(jsonsrc) if _.endswith('.json')])
wsis = sorted([_ for _ in os.listdir(wsisrc) if _.endswith('.ndpi')])
m1s = sorted([_ for _ in os.listdir(m1src) if _.endswith('.tif')])
m2s = [_ for _ in os.listdir(m2src)]

cellcounts = []
zlevels = []
sizes = []

for num, json in enumerate(jsons):
    zlevel = int(json[-9:-5])
    jsonpth = os.path.join(jsonsrc,json)
    wsipth = os.path.join(wsisrc, wsis[num])
    m1pth = os.path.join(m1src, m1s[num])
    m2pth = os.path.join(m2src, m2s[num])


    print(zlevel)
    print(jsonpth)
    print(wsipth)
    print(m1pth)
    print(m2pth)

    start = time()
    json = pd.read_json(jsonpth)
    end = time()
    print('Loading json took: {}'.format(round(end-start)))

    start = time()
    m1 = Image.open(m1pth)
    m1h, m1w = m1.size

    wsi = OpenSlide(wsipth)
    wsih, wsiw = wsi.dimensions

    start = time()
    json['type'] = json['centroid'].apply(lambda row: cellclass(row,dl,rsfw_wsi2dl,rsfh_wsi2dl))

    ECMclass=6
    jsonECM = json[json.type==ECMclass]

    json = jsonECM
    json['Area'] = json['contour'].apply(lambda row: cntarea(row))
    json['Perimeter'] = json['contour'].apply(lambda row: cntperi(row))
    json['Circularity'] = 4 * np.pi * json['Area'] / json['Perimeter'] ** 2
    json['MA'] = json['contour'].apply(lambda row: cntMA(row))
    json[['MA', 'ma', 'orientation']] = pd.DataFrame(json.MA.tolist())
    json['AspectRatio'] = json['MA'] / json['ma']
    jsonRound = json[json['AspectRatio']<2]

    cellcounts.append(len(jsonRound))
    zlevels.append(zlevel)
    sizes.append(json['Area'].mean())
    print('Classifying and quantifying nuclei took: {}'.format(round(end-start)))


1
\\10.99.68.178\andreex\data\monkey fetus\gestational 40\StarDist_11_21_23\json\monkey_fetus_40_0001.json
\\10.99.68.178\andreex\data\monkey fetus\gestational 40\monkey_fetus_40_0001.ndpi
\\10.99.68.178\andreex\data\monkey fetus\gestational 40\5x\cropped_images\classification_MODEL1_6_21_2023_all_annotations\monkey_fetus_40_0001.tif
\\10.99.68.178\andreex\data\monkey fetus\gestational 40\all_classifications\classification_6_13_2023_adrenal_donald
Loading json took: 1


NameError: name 'dl' is not defined

In [44]:
json['centroid'][0][0][0]

1928

In [7]:
df = {'zlevel':zlevels,'cellcounts':cellcounts,'cellsize':sizes}
df = pd.DataFrame(df)
df.to_csv(os.path.join(jsonsrc,'stat.csv'))

In [ ]:
#VISUALIZATION
centroidmap = np.zeros([dlh,dlw])
marksize = 10
for index, row in jsonRound.iterrows():
    xy = [round(_/rsfw_wsi2dl) for _ in  row.centroid]
    centroidmap[xy[1]-marksize:xy[1]+marksize,xy[0]-marksize:xy[0]+marksize]=1

Image.fromarray(centroidmap.astype(np.uint8)).save('tmp.png')